In [1]:
import sys
import os, glob
from netCDF4 import Dataset as open_ncfile
import matplotlib.pyplot as plt
from maps_matplot_lib import defVarmme, zon_2Dz, custom_div_cmap, modelagree, zonal_2D
import numpy as np
import xarray as xr
from functions import lag_linregress_3D
from scipy.stats import pearsonr

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Read every run and compute 1950-2008 trend

In [3]:
# == Read models ==
from modelsDef import defModels

models = defModels()

indir_histrcp85 = '/home/ysilvy/Density_bining/Yona_analysis/data/hist_rcp85/'
time = np.arange(1950,2101)

# Choose where to end time series
y2 = 2008 
trend_models = [0]*11
vart_models = [0]*11
model_names = ['']*11
bowl_models = [0]*11
imod=0

for i, model in enumerate(models):
#i=2; model=models[i]
    # Read hist+rcp85 files
    listruns = sorted(glob.glob(indir_histrcp85 + 'cmip5.' + model['name'] + '.' + '*zon2D.nc'))
    listruns1d = sorted(glob.glob(indir_histrcp85 + 'cmip5.' + model['name'] + '.' + '*zon1D.nc'))
    nruns = len(listruns)
    if nruns != 0:
        print('Working on', model['name'])
        # Index of common time interval
        tstart = model['props'][2]
        tend = model['props'][3] #2005
        # Loop over number of runs
        for k in range(nruns):
            namefile = os.path.basename(listruns[k])
            run_nb = namefile.split('.')[3]
            print('    . run number', k, run_nb)

            fhrcp = xr.open_dataset(listruns[k],decode_times=False)
            varkt = fhrcp.isonso.isel(time=slice(tstart+89,tend+95)) #1950-2100
            varkt['time'] = time
            varkt = varkt.sel(time=slice(1950,y2))
            fhrcp1d = xr.open_dataset(listruns1d[k],decode_times=False)
            bowlk = fhrcp1d.ptopsigma.isel(time=slice(tstart+89,tend+95))
            bowlk['time'] = time
            bowlk = bowlk.sel(time=slice(y2-10,y2)).mean(dim='time')

            # Compute trend at each grid point
            covk,cork,slopek,interceptk,pvalk,stderrk = lag_linregress_3D(x=varkt.time,y=varkt)
            trendk = slopek*50 #len(varkt.time) 
            
            # Add dimension to stack members
            trendk['members'] = run_nb
            trendk = trendk.expand_dims('members') 
            varkt['members'] = run_nb
            varkt = varkt.expand_dims('members') 
            bowlk['members'] = run_nb
            bowlk = bowlk.expand_dims('members') 
            if k==0:
                trend_mod = trendk
                bowl_mod = bowlk
                vart_mod = varkt
            else:
                trend_mod = xr.concat([trend_mod,trendk],dim='members')
                bowl_mod = xr.concat([bowl_mod,bowlk],dim='members')
                vart_mod = xr.concat([vart_mod,varkt],dim='members')
            
        trend_models[imod] = trend_mod
        model_names[imod] = model['name']
        bowl_models[imod] = bowl_mod
        vart_models[imod] = vart_mod
        imod=imod+1


Working on CanESM2
    . run number 0 r1i1p1
    . run number 1 r2i1p1
    . run number 2 r3i1p1
    . run number 3 r4i1p1
    . run number 4 r5i1p1
Working on CCSM4
    . run number 0 r1i1p1
    . run number 1 r2i1p1
    . run number 2 r3i1p1
    . run number 3 r4i1p1
    . run number 4 r5i1p1
    . run number 5 r6i1p1
Working on CESM1-CAM5
    . run number 0 r1i1p1
    . run number 1 r2i1p1
    . run number 2 r3i1p1
Working on CSIRO-Mk3-6-0
    . run number 0 r1i1p1
    . run number 1 r2i1p1
    . run number 2 r3i1p1
    . run number 3 r4i1p1
    . run number 4 r5i1p1
    . run number 5 r6i1p1
    . run number 6 r7i1p1
    . run number 7 r8i1p1
    . run number 8 r9i1p1
Working on FGOALS-g2
    . run number 0 r1i1p1
Working on GISS-E2-R
    . run number 0 r1i1p1
    . run number 1 r2i1p1
    . run number 2 r2i1p3
Working on HadGEM2-ES
    . run number 0 r1i1p1
    . run number 1 r3i1p1
Working on IPSL-CM5A-LR
    . run number 0 r2i1p1
    . run number 1 r3i1p1
    . run number 2 r4i1

## Compute ensemble means

In [4]:
# Dimensions
basin = trend_models[0].basin; basinN = len(basin)
lat = trend_models[0].latitude; latN = len(lat)
lev = trend_models[0].lev; levN = len(lev)
time = vart_models[0].time; timeN = len(time)
# Initialize xarrays
trend_means = xr.DataArray(np.zeros((len(trend_models),basinN,levN,latN)),dims=['models','basin','lev','latitude'],
                             coords=[model_names,basin,lev,lat])
bowl_means = xr.DataArray(np.zeros((len(trend_models),basinN,latN)),dims=['models','basin','latitude'],
                             coords=[model_names,basin,lat])
vart_means = xr.DataArray(np.zeros((len(trend_models),timeN,basinN,levN,latN)),dims=['models','time','basin','lev','latitude'],
                             coords=[model_names,time,basin,lev,lat])
# Loop on models to compute ensemble means
for i in range(len(trend_models)):
    trend_means[i,:,:,:] = trend_models[i].mean(dim='members')
    bowl_means[i,:,:] = bowl_models[i].mean(dim='members')
    vart_means[i,:,:,:,:] = vart_models[i].mean(dim='members')

## Compute multi-model means 

In [5]:
trend_mmm = trend_means.mean(dim='models') # mmm of trends
bowl_mmm = bowl_means.mean(dim='models')
vart_mmm = vart_means.mean(dim='models') # mmm of temporal data

## Mask above the bowl

In [6]:
# -- Now mask points in the bowl
for ilat in range(latN):
    if np.isnan(bowl_mmm[1,ilat]) == False :
        inda = np.ma.nonzero(bowl_mmm[1,ilat]>=lev)[0]
        trend_mmm[1,inda,ilat] = np.nan
        trend_means[:,1,inda,ilat] = np.nan
    if np.isnan(bowl_mmm[2,ilat]) == False :
        indp = np.ma.nonzero(bowl_mmm[2,ilat]>=lev)[0]
        trend_mmm[2,indp,ilat] = np.nan
        trend_means[:,2,indp,ilat] = np.nan
    if np.isnan(bowl_mmm[3,ilat]) == False :
        indi = np.ma.nonzero(bowl_mmm[3,ilat]>=lev)[0]
        trend_mmm[3,indi,ilat] = np.nan
        trend_means[:,3,indi,ilat] = np.nan

## Now compute mmm hist-histNat [1985-2005]

In [7]:
indirhn = '/data/ysilvy/Density_binning/mme_histNat/'
filehn_2d = 'cmip5.multimodel_Nat_rcp85.historicalNat.ensm.an.ocn.Omon.density_zon2D.nc'
filehn_1d = 'cmip5.multimodel_Nat_rcp85.historicalNat.ensm.an.ocn.Omon.density_zon1D.nc'
fhn2d = xr.open_dataset(indirhn+filehn_2d,decode_times=False)
fhn1d = xr.open_dataset(indirhn+filehn_1d,decode_times=False)

varhn = fhn2d.isonso.isel(time=slice(125,145))

In [8]:
var_change = vart_mmm.sel(time=slice(1985,2005)).mean(dim='time') - varhn.mean(dim='time')

In [9]:
# -- Mask above bowl
for ilat in range(latN):
    if np.isnan(bowl_mmm[1,ilat]) == False :
        inda = np.ma.nonzero(bowl_mmm[1,ilat]>=lev)[0]
        var_change[1,inda,ilat] = np.nan
    if np.isnan(bowl_mmm[2,ilat]) == False :
        indp = np.ma.nonzero(bowl_mmm[2,ilat]>=lev)[0]
        var_change[2,indp,ilat] = np.nan
    if np.isnan(bowl_mmm[3,ilat]) == False :
        indi = np.ma.nonzero(bowl_mmm[3,ilat]>=lev)[0]
        var_change[3,indi,ilat] = np.nan

## Compute correlation 

In [45]:
# -- Stack all dimensions including basins
x = trend_mmm.sel(basin=slice(1,3)).stack(z=('basin','lev','latitude'))
y = var_change.sel(basin=slice(1,3)).stack(z=('basin','lev','latitude'))

In [46]:
# Remove nans to compute correlation coefficient
ixnonan = ~np.isnan(x)
iynonan= ~np.isnan(y)
ixynonan = ixnonan & iynonan
xnonan = x[ixynonan]
ynonan = y[ixynonan]

In [47]:
corr = pearsonr(xnonan,ynonan)

In [48]:
corr

(0.8810002320691286, 0.0)

In [51]:
# -- Each basin zonal mean 
for i in range(4):
    xtot = trend_mmm.sel(basin=i).stack(z=('lev','latitude'))
    ytot = var_change.sel(basin=i).stack(z=('lev','latitude'))
    # Remove nans to compute correlation coefficient
    ixnonan = ~np.isnan(xtot)
    iynonan= ~np.isnan(ytot)
    ixynonan = ixnonan & iynonan
    xnonan = xtot[ixynonan]
    ynonan = ytot[ixynonan]
    corr = pearsonr(xnonan,ynonan)
    print(i,corr)

0 (0.7265332373300097, 0.0)
1 (0.871016107243001, 0.0)
2 (0.8732607844616733, 0.0)
3 (0.8847327596184282, 0.0)
